# Self-Driving Car Engineer Nanodegree


## Project: **Behavioral Cloning** 
***

I have decided to use a Jupyter notebook like the previous projects to do the initial work atleast.

---

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [23]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import cv2
import math
import os
import csv
import random
from scipy import signal
import tensorflow as tf
from glob import glob

from keras.utils import np_utils
from keras.preprocessing import image
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras import optimizers

%matplotlib inline

## Read in Images

In [ ]:
lines = []

with open('../data/data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

#print(lines[0])
#print(lines[1])
#print(lines[2:4])
        
images = []
measurements = []

for line in lines[1:]:
    #print(line)
    #print("")
    
    source_path = line[0]
    #print(source_path)
    filename = source_path.split('/')[-1]
    current_path = '../data/data/IMG/' + filename
    #print(current_path)
    image = cv2.imread(current_path)
    images.append(image)

    source_path = line[1]
    #print(source_path)
    filename = source_path.split('/')[-1]
    current_path = '../data/data/IMG/' + filename
    #print(current_path)
    image = cv2.imread(current_path)
    images.append(image)

    source_path = line[2]
    #print(source_path)
    filename = source_path.split('/')[-1]
    current_path = '../data/data/IMG/' + filename
    #print(current_path)
    image = cv2.imread(current_path)
    images.append(image)

    #print("")
    
    measurement = float(line[3])
    measurements.append(measurement)
    
    #print(measurements)



## Setup Training Data

In [ ]:
    X_train = np.array(images)
    y_train = np.array(measurements)

## Nvidia Deep Learning Model

In [ ]:
model = Sequential()

model.add(Flatten(input_shape = (160, 320, 3)))
model.add(Dense(1))

model.summary()

## Compile The Model